# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# Surface Water Flow with Channel Flow (CHF) Model

In this example, the Channel Flow (CHF) Model is used to simulate stream flow in Mahoning Creek, near Punxsutawney, PA.  This problem is based on a tutorial designed for the [HEC-HMS software](https://www.hec.usace.army.mil/confluence/hmsdocs/hmsguides/applying-reach-routing-methods-within-hec-hms/introduction-to-the-channel-routing-tutorials).

<img src="./mahoning_creek_watershed.png" alt="drawing" width="400"/>

In [ ]:
# imports
from pathlib import Path

import flopy
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import PatchCollection

## Hydrographs

The data for this problem is contained in `mahoning_data.py`.  As shown in the figure below, there is an inflow hydrograph and and outflow hydrograph.  

<img src="./hydrographs.png" alt="drawing" width="400"/>

According to the tutorial, the inflow hydrograph is synthetic.  The outflow hydrograph was simulated by the HEC-RAS software.

In [ ]:
# load the hydrograph data
import mahoning_data

data = mahoning_data.get_data()
punx_inflow_hydrograph = data["inflow_hydrograph"]
punx_hec_hms_outflow = data["hec_hms_outflow"]
punx_obs_outflow = data["obs_outflow"]
sample_times = data["sample_times"]
dt = 15 * 60  # 15 mins converted to seconds
total_time = punx_inflow_hydrograph.shape[0] * dt
print(f"{total_time=} seconds")

In [ ]:
# plot the hydrographs
sec_to_min = 1.0 / 60.0
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(
    sample_times * sec_to_min, punx_inflow_hydrograph, "b-", label="inflow"
)
ax.plot(sample_times * sec_to_min, punx_obs_outflow, "r-", label="outflow")
ax.set_xlabel("time, in minutes")
ax.set_ylabel("flow, in cms")
ax.legend()
ax.set_title("Mahoning Creek")

## Cross Section

The tutorial provides three different cross sections.  Here we use the cross section at Punxsutawney for the entire creek.

<img src="./mahoning_cross_sections.png" alt="drawing" width="400"/>

In [ ]:
# This cell loads the cross section data into a form
# that can used to construct the Cross Section (CXS)
# Package for the MODFLOW SWF Model.  As part of the
# exercise you will be adjusting the channel_n and
# bank_n in order to match the outflow hydrograph.

txt = "Punxsutawney Cross Section"
section_name = "full"  # or "8point"
channel_n = 0.03
bank_n = 0.05
cross_section_data = mahoning_data.get_cross_section_data(
    section_name=section_name,
    channel_n=channel_n,
    bank_n=bank_n,
)

# This data has metric units
x = cross_section_data["x"]
h = cross_section_data["h"]
r = cross_section_data["r"]

# plot the cross section with line widths scaled by mannigns value
from matplotlib import cm

for i in range(x.shape[0] - 1):
    plt.plot(
        x[i : i + 2],
        h[i : i + 2],
        lw=r[i] * 50,
        color="brown",
        label=f"rough={r[i]}",
    )
plt.plot(x, h, marker="o", mec="brown", mfc="white", lw=0.0)
plt.xlabel("station, in m")
plt.ylabel("elevation, in m")
plt.title(txt)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

## Setup Model Parameters

In [ ]:
# Set dx and dt
stream_length = 68861.0 / 3.2808  # converted to  meters
total_time = sample_times[-1]
ncells = 8 * 1
nstp = 72 * 1
dx = stream_length / ncells
dt = total_time / nstp
celerity = 5.0 / 3.2808  # convert 5 feet per second to meters per second
print(f"{stream_length=:0.2f} meters")
print(f"{total_time=} seconds")
print(f"{ncells=}")
print(f"{nstp=}")
print(f"{dx=:0.2f} meters")
print(f"{dt=} seconds")
print(f"desired dx based on courant = {celerity * dt} meters")
print(f"desired dt based on courant = {dx / celerity} seconds")

In [ ]:
# set up reach bottom
dz = 82.0 / 3.2808  # elevation change from upstream to downstream
slope = dz / stream_length
print(f"{dz=} meters")
print(f"{slope=} meters per meter")
x = np.linspace(dx / 2, stream_length - dx / 2, ncells)
reach_bottom = x * slope
reach_bottom = reach_bottom[::-1]

# set up vertices
vertices = []
vertices = [[j, j * dx, 0.0] for j in range(ncells + 1)]
cell2d = []
for j in range(ncells):
    cell2d.append([j, 0.5, 2, j, j + 1])

## Build and Run MODFLOW Model

In [ ]:
# make modflow model
sim_ws = Path("./mahoning_mf6")
name = "mahoning"
sim = flopy.mf6.MFSimulation(
    sim_name=name,
    version="mf6",
    sim_ws=sim_ws,
    memory_print_option="summary",
    continue_=False,
)
tdis = flopy.mf6.ModflowTdis(
    sim, nper=1, perioddata=[(total_time, nstp, 1.0)], time_units="seconds"
)

# adaptive time stepping
if False:
    # set dt0, dtmin, dtmax, dtadj, dtfailadj
    dt0 = 60 * 1.0  # 1 min (in seconds)
    dtmin = 1.0  # (in seconds)
    dtmax = 60 * 30.0  # 30 min (in seconds)
    dtadj = 2.0
    dtfailadj = 5.0
    ats_filerecord = name + ".ats"
    atsperiod = [
        (0, dt0, dtmin, dtmax, dtadj, dtfailadj),
    ]
    tdis.ats.initialize(
        maxats=len(atsperiod),
        perioddata=atsperiod,
        filename=ats_filerecord,
    )


dvclose = 1.0e-8
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="summary",
    outer_dvclose=dvclose,
    outer_maximum=100,
    under_relaxation="DBD",
    under_relaxation_theta=0.95,
    under_relaxation_kappa=0.0001,
    under_relaxation_gamma=0.0,
    under_relaxation_momentum=0.0,
    inner_maximum=20,
    inner_dvclose=dvclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=0.97,
    # backtracking_number=5,
    # backtracking_tolerance=1.0,
    # backtracking_reduction_factor=0.3,
    # backtracking_residual_limit=100.0,
    csv_outer_output_filerecord=f"{name}.ims.outer.csv",
)

chf = flopy.mf6.ModflowChf(
    sim,
    modelname=name,
    save_flows=True,
    newtonoptions=True,
)
disv1d = flopy.mf6.ModflowChfdisv1D(
    chf,
    nodes=ncells,
    bottom=reach_bottom,
    width=1.0,
    vertices=vertices,
    cell1d=cell2d,
    idomain=1,
)
ic = flopy.mf6.ModflowChfic(chf, strt=reach_bottom + 1.0)
slope = 0.0012
dfw = flopy.mf6.ModflowChfdfw(
    chf,
    print_flows=True,
    save_flows=True,
    manningsn=1.0,
    idcxs=0,
)
sto = flopy.mf6.ModflowChfsto(
    chf,
    save_flows=True,
)
xfraction = cross_section_data["x"]
height = cross_section_data["h"]
mannfraction = cross_section_data["r"]
npoints = xfraction.shape[0]
cxsdata = list(zip(xfraction, height, mannfraction))
cxs = flopy.mf6.ModflowChfcxs(
    chf,
    nsections=1,
    npoints=npoints,
    packagedata=[(0, npoints)],
    crosssectiondata=cxsdata,
)
flw = flopy.mf6.ModflowChfflw(
    chf,
    maxbound=1,
    print_input=True,
    print_flows=True,
    stress_period_data=[(0, "inflow")],
)
fname = f"{name}.flw.ts"
flw.ts.initialize(
    filename=fname,
    timeseries=list(zip(sample_times, punx_inflow_hydrograph)),
    time_series_namerecord=["inflow"],
    interpolation_methodrecord=["linearend"],
)

fname = f"{name}.zdg.obs.csv"
zdg_obs = {
    fname: [
        ("OUTFLOW", "ZDG", (ncells - 1,)),
    ],
    "digits": 10,
}
idcxs = 0  # use cross section 0
width = 1.0
slope = slope
rough = 1.0
spd = [((ncells - 1,), idcxs, width, slope, rough)]
zdg = flopy.mf6.ModflowChfzdg(
    chf,
    observations=zdg_obs,
    print_input=True,
    maxbound=len(spd),
    stress_period_data=spd,
)

oc = flopy.mf6.ModflowChfoc(
    chf,
    budget_filerecord=f"{name}.bud",
    stage_filerecord=f"{name}.stage",
    saverecord=[
        ("STAGE", "ALL"),
        ("BUDGET", "ALL"),
    ],
    printrecord=[
        ("STAGE", "LAST"),
        ("BUDGET", "ALL"),
    ],
)
obs_data = {
    f"{name}.obs.csv": [
        ("STAGE1", "STAGE", (0,)),
        (f"STAGE{ncells}", "STAGE", (ncells - 1,)),
    ],
}
obs_package = flopy.mf6.ModflowUtlobs(
    chf,
    filename=f"{name}.obs",
    digits=10,
    print_input=True,
    continuous=obs_data,
)

sim.write_simulation()
success, buff = sim.run_simulation(silent=False)
assert success, buff

## Post Process

In [ ]:
fpth = Path(sim_ws) / f"{name}.zdg.obs.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")
sim_outflow = -obsvals["OUTFLOW"]
sim_times = obsvals["time"]
fix, ax = plt.subplots(figsize=(6, 3))
ax.plot(sample_times, punx_inflow_hydrograph, "b-", label="inflow")
ax.plot(sample_times, punx_obs_outflow, "r-", label="measured outflow")
ax.plot(
    sim_times,
    sim_outflow,
    mew=0.5,
    ms=3.0,
    marker="o",
    mfc="white",
    mec="red",
    lw=0.0,
    label="simulated outflow",
)
ax.legend()
ax.set_ylabel("flow, in $m^3/s$")
ax.set_xlabel("time, in seconds")

In [ ]:
fpth = Path(sim_ws) / f"{name}.obs.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")
sim_times = obsvals["time"]
sim_stage1 = obsvals["STAGE1"]
sim_stagen = obsvals[f"STAGE{ncells}"]
sim_depth1 = sim_stage1 - reach_bottom[0]
sim_depthn = sim_stagen - reach_bottom[ncells - 1]
fix, ax = plt.subplots(figsize=(6, 3))
ax.plot(
    sim_times,
    sim_depth1,
    mew=0.5,
    ms=3.0,
    marker="o",
    mfc="blue",
    lw=0.0,
    label="MODFLOW Stage 1",
)
ax.plot(
    sim_times,
    sim_depthn,
    mew=0.5,
    ms=3.0,
    marker="o",
    mfc="red",
    lw=0.0,
    label=f"MODFLOW Stage {ncells}",
)
ax.legend()
ax.set_ylabel("water depth, in meters")
ax.set_xlabel("time, in seconds")

## Exercise

The goal of this exercise is to calibrate the Mahoning Creek model by adjusting the Manning's roughness coefficients.  With some simple adjustments to the channel and floodplain roughness values, you should be able to match the simulated outflow with the known solution.  The channel roughness should be less than the bank roughness.  

1.  Adjust the channel and bank (floodplain) roughness until the simulated outflow matches the measured outflow.  Consult [this table](https://www.fsl.orst.edu/geowater/FX3/help/8_Hydraulic_Reference/Mannings_n_Tables.htm) for reasonable values for main channel and floodplain roughness coefficients.

2.  Try increasing the spatial and temporal resolution to get a more refined answer.  This can be done by changing the values for `ncells` and `nstp`.

3.  Try turning on the adaptive time stepping.

## Create Animation

In [ ]:
def get_x_positions(vertices, cell2d):
    x = []
    for cell_data in cell2d:
        icell_number = cell_data[0]
        vstart = cell_data[-2]
        vend = cell_data[-1]
        x.extend(vertices[[vstart, vend]]["xv"])
    return x


def get_z_positions(zdata, vertices, cell2d):
    z = []
    for cell_data in cell2d:
        icell_number = cell_data[0]
        vstart = cell_data[-2]
        vend = cell_data[-1]
        z.extend(2 * [zdata[icell_number]])
    return z


def plot_channel_bottom(ax, vertices, cell2d, reach_bottom):
    x = get_x_positions(vertices, cell2d)
    z = get_z_positions(reach_bottom, vertices, cell2d)
    ax.plot(x, z, "k--", lw=1)
    return


def get_patch_bounds(stage, vertices, cell2d, reach_bottom):
    """
    Create information for rectangular patches that span
    from stage to reach_bottom

    """
    xy = []
    width = []
    height = []
    for cell_data in cell2d:
        icell_number = cell_data[0]
        vstart = cell_data[-2]
        vend = cell_data[-1]
        xv0 = vertices[vstart]["xv"]
        xv1 = vertices[vend]["xv"]
        ztop = stage[icell_number]
        zbot = reach_bottom[icell_number]
        xy.append((xv0, zbot))
        width.append(xv1 - xv0)
        height.append(max(ztop - zbot, 0.0))
    return xy, width, height


def get_patch_list(xy_list, width_list, height_list):
    from matplotlib.patches import Rectangle

    pc_list = []
    for xy, width, height in zip(xy_list, width_list, height_list):
        pc_list.append(Rectangle(xy, width, height))
    return pc_list

In [ ]:
# Uncomment below to generate animation

# fig, ax = plt.subplots(figsize=(8, 3))
# ax.set_xlabel(r"x")
# ax.set_ylabel(r"z")
# ax.set_ylim(0, 35)
# title = ax.set_title(f"{0} percent completed")

# # plot persistent items
# vertices = chf.disv1d.vertices.get_data()
# cell1d = chf.disv1d.cell1d.get_data()
# reach_bottom = chf.disv1d.bottom.get_data()
# plot_channel_bottom(ax, vertices, cell1d, reach_bottom)

# patch_data = get_patch_bounds(reach_bottom, vertices, cell1d, reach_bottom)
# pc_list = get_patch_list(*patch_data)
# pc = PatchCollection(pc_list, facecolor="blue", alpha=0.5, edgecolor="none")
# ax.add_collection(pc)

# fpth = sim_ws / f"{chf.name}.stage"
# sobj = flopy.utils.HeadFile(fpth, precision="double", text="STAGE")
# times = sobj.get_times()
# stage_all = sobj.get_alldata().squeeze()


# def animate(i):
#     stage = stage_all[i]

#     patch_data = get_patch_bounds(stage, vertices, cell1d, reach_bottom)
#     pc_list = get_patch_list(*patch_data)

#     pc.set_paths(pc_list)
#     title = ax.set_title(
#         f"{times[i] / times[-1] * 100.:0.2f} percent completed"
#     )
#     return


# import matplotlib.animation

# ani = matplotlib.animation.FuncAnimation(
#     fig, animate, frames=stage_all.shape[0]
# )

# plt.close()
# from IPython.display import HTML

# HTML(ani.to_jshtml())